# Periodogram of a periodic acoustic signal

Before working on real stars, we need to better understand the meaning of periodogram. To do this, let's start by analysing the frequencies that make up the underlying periodic signal. Don't worry, you won't have to do the calculations yourself! The ```periodogram``` package contained in the ```scipy.signal``` library will take care of that. Finally, we will transform the content of this signal into audible sound using the ```Audio``` library of```IPython.display```.

In [ ]:
import numpy as np
from scipy.signal import periodogram
import plotly.graph_objects as go
from IPython.display import Audio

Imagine you are in a recording studio. To impress your musician friends, you propose a challenge: by recording only about a second of their new song (alternative music!), you will be able to tell them the notes it contains and their intensity.

In [ ]:
#Sound corresponding to about 73 deciBel (singing person)
N = 5e+4
fs = 4.41e+4 #audio file sampling frequency
time = np.arange(N) / fs # in seconds
flux = 1e+6/101325.*(0.12*np.sin(2*np.pi*261.6256*time)+0.12*np.sin(2*np.pi*329.6276*time)+0.12*np.sin(2*np.pi*391.9954*time)+0.12*np.sin(2*np.pi*65.40639*time)+0.5*0.12*np.sin(2*np.pi*73.41619*time)+0.33*0.12*np.sin(2*np.pi*164.8138*time)+0.25*0.12*np.sin(2*np.pi*440.*time)+0.12*np.sin(2*np.pi*1046.502*time))

fig = go.Figure()

fig.add_scatter(x=time, y=flux, mode='lines', marker=dict(size=1.5, color="black") )

fig.update_layout(title="Sound pressure of the audio track", xaxis_title="Time [s]", yaxis_title="Sound pressure [ppm]")#, xaxis_range=[0,1])


In [ ]:
Audio(flux, rate=fs)

As you can see (and hear), the signal does not have a single period (i.e., it is not composed of a single musical note). The best way to find the notes used is through a periodogram, that is, by analysing the power spectral density (PSD) of each individual frequency.

In [ ]:
f, psd = periodogram(flux,fs=fs,scaling='density')

fig = go.Figure()

fig.add_scatter(x=f, y=psd, mode='lines', marker=dict(size=1.5, color="black") )

fig.update_layout(title="Periodogramma", xaxis_title="$\mbox{Frequency} \, [ \mbox{Hz}]$", yaxis_title="$\mbox{PSD} \, [\mbox{ppm}^2/  \mbox{Hz}]$")


It's done! There are clear peaks centred at around 65 Hz (C of the second octave), 73 Hz (D of the second octave), 165 Hz (E of the third octave), 262 Hz (C of the fourth octave), 330 Hz (E of the fourth octave), 391 Hz (G of the fourth octave), 440 Hz (A of the fourth octave) and finally 1047 Hz (C of the sixth octave).

How can we improve the estimation of frequencies? By increasing the number of observations, i.e. by listening to the signal for as long as possible.

# Exercise: asteroseismology of red giant stars with *Kepler*

Now let's see how to download the packages needed to draw the light curve of a red giant star.

First of all, we will need a library called ```lightkurve```. This will allow us to download data on a red giant observed for about 4 years by *Kepler*.

First, let's install this library on our server.

In [ ]:
!pip install lightkurve

In [ ]:
import numpy as np
import lightkurve as lk
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib notebook

Now that ```Python``` can understand us, let's look for the red giant star called KIC2436824.

In [ ]:
search_result = lk.search_lightcurve('KIC2436824', author='Kepler', cadence='long')


In [ ]:
search_result

Great! Now we are ready to download this table to our server and construct the light curve (which we will call ```lcobs```).

In [ ]:
lcobs = search_result.download_all().stitch().remove_outliers()

In [ ]:
lcobs.plot()

As you can see, ```lightkurve``` itself allows us to plot the light curve. However, we want to interact with the data, so we create one with ```plotly``` using time (```lcobs.time.value```) and flux (```lcobs.flux.value```). In addition, we will add the flux error (```lcobs.flux_err.value```).

In [ ]:
fig = go.Figure()

fig.add_scatter(x=lcobs.time.value, y=lcobs.flux.value, mode='markers', marker=dict(size=3, color="black"), error_y=dict(type = "data", array=lcobs.flux_err.value))

fig.update_layout(title="Light curve", xaxis_title="Time [days]", yaxis_title="Normalized flux")


Now let's construct the periodogram of this light curve and call it ```pobs```. Let's also construct its “smooth” version, i.e. a version of the periodogram that highlights the dominant frequencies.

In [ ]:
pobs = lcobs.normalize('ppm').to_periodogram(ls_method ='auto',normalization='psd')

In [ ]:
pobs_smooth = pobs.smooth(filter_width=0.1)

In [ ]:
ax = pobs.plot()
pobs_smooth.plot(ax=ax,scale='log',label='Smooth',color='red')
plt.show()

As before, ```lightkurve``` itself creates a plot of the two periodograms. The regular one is in black, and the version that emphasises the dominant frequencies is in red. What do you notice from the graph?

To see what is happening more clearly, we create the plot with ```plotly``` using the frequencies (```pobs.frequency.value```) and the power density (```pobs.power.value```). In addition, we will add the frequencies (```pobs_smooth.frequency.value```) and power density (```pobs_smooth.power.value```) of the smoother model for comparison.

In [ ]:
fig = go.Figure()

fig.add_scatter(x=pobs.frequency.value, y=pobs.power.value, mode='lines', marker=dict(color="black"))

fig.add_scatter(x=pobs_smooth.frequency.value, y=pobs_smooth.power.value, mode='lines', marker=dict(color="red"))

fig.update_layout(title="Periodogram",
                  xaxis_title="$\mbox{Frequency} \, [\mu \mbox{Hz}]$",
                  yaxis_title="$ \mbox{Power} \, [\mbox{ppm}^2/\mu \mbox{Hz}]$",
                  xaxis_range=[20,50])


What peaks do you notice? Are they all equally high? Are they all equally spaced?

Now let's analyse the spectrum using the signal-to-noise ratio (SNR), i.e. by removing any form of noise from the periodogram. We will call this new periodogram ```pobs_flatten```.

In [ ]:
pobs_flatten = pobs.flatten()

In [ ]:
pobs_flatten.plot()

Excellent! Now it is even clearer why we previously selected a range between 20 and 50 $\mu$Hz: this is the region where it is possible to observe the oscillations produced by the star itself.

As before, let's make our plot with ```plotly``` using the frequencies (```pobs_flatten.frequency.value```) and the SNR (```pobs_flatten.power.value```).

In [ ]:
fig = go.Figure()

fig.add_scatter(x=pobs_flatten.frequency.value, y=pobs_flatten.power.value, mode='lines', marker=dict(color="black"))
fig.update_layout(title="Periodogram",
                  xaxis_title="$\mbox{Frequency} \, [\mu \mbox{Hz}]$",
                  yaxis_title="Signal to Noise (SNR)",
                  xaxis_range=[20,50])

At this point, we have almost all the information we need to find the mass and radius of the star. The last important piece of information can be found in the literature and is the effective temperature, which in this case is 4343 K.

Is this star hotter or cooler than the Sun? Is it more or less massive? Where might it be located on the HR diagram?

In [ ]:
pobs_seismology = pobs_flatten.to_seismology()
print(f'Numax can be estimated as {pobs_seismology.estimate_numax()} respect to 34.03')
print(f'dnu can be estimated as {pobs_seismology.estimate_deltanu():.3f} respect to 3.87')
print(f'The radius can be estimated as {pobs_seismology.estimate_radius(teff=4343):.3f}')
print(f'The mass can be estimated as {pobs_seismology.estimate_mass(teff=4343):.3f}')
print(f'The luminosity can be estimated as {(4343/5777)**4*pobs_seismology.estimate_radius(teff=4343).value**2:.1f} L_sol')


Now let's transform the frequencies present in the light curve into audible sound (i.e., let's start from $\mu$Hz to hundreds of Hz). To do this, insert the 16 more dominant frequencies you see in the previous plot, starting with the smallest, in place of the ```...``` .

For example, above we see a peak at around 25.4 $\mu$Hz: to make it audible, we multiply it by 10 million, obtaining a sound at 254 Hz. This will be the first frequency to insert below.

In [ ]:
# Sound corresponding to about 73 deciBel (singing person)
N2 = 7e+4
time2 = np.arange(N2) / fs # in seconds
flux2 = 1e+6/101325.*0.12*(
    0.25*np.sin(2*np.pi*254.*time2)
    +0.25*np.sin(2*np.pi*265.*time2)
    +0.25*np.sin(2*np.pi*269.*time2)
    +0.5*np.sin(2*np.pi*289.*time2)
    +0.5*np.sin(2*np.pi*301.*time2)
    +0.5*np.sin(2*np.pi*307.*time2)
    +np.sin(2*np.pi*328.*time2)
    +np.sin(2*np.pi*340.*time2)
    +np.sin(2*np.pi*345.*time2)
    +0.7*np.sin(2*np.pi*366.*time2)
    +0.33*np.sin(2*np.pi*378.*time2)
    +0.33*np.sin(2*np.pi*383.*time2)
    +0.33*np.sin(2*np.pi*404.*time2)
    +0.33*np.sin(2*np.pi*418.*time2)
    +0.33*np.sin(2*np.pi*424.*time2)
    +0.33*np.sin(2*np.pi*445.*time2)
    )

In [ ]:
fig = go.Figure()

fig.add_scatter(x=time2, y=flux2, mode='lines', marker=dict(size=1.5, color="black") )

fig.update_layout(title="Pressione sonora della traccia audio", xaxis_title="Time [s]", yaxis_title="Sound pressure [ppm]")#, xaxis_range=[0,1])


In [ ]:
Audio(flux2, rate=fs)